<a href="https://colab.research.google.com/github/mohamad1371/multi_class-image-classification-with-inceptionResnetV2/blob/main/image_classification_with_InceptionResNetV2_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

!unzip "/content/drive/MyDrive/dataset.zip" -d "/content/unzip_data/"

In [ ]:
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Flatten, RepeatVector, Reshape, ZeroPadding2D, Input, Dense

In [ ]:
def feature_extractor(inputs):
    feature_extractor_layers = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
        input_shape=(299, 299, 3),
        include_top = False,
        weights = 'imagenet')
    feature_extractor_layers.trainable = False
    feature_extractor_layers = feature_extractor_layers(inputs)

    return feature_extractor_layers    

In [ ]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(2048, activation = "relu")(x)
    x = tf.keras.layers.Dense(1024, activation = "relu")(x)
    x = tf.keras.layers.Dense(512, activation = "relu")(x)
    x = tf.keras.layers.Dense(10, activation = "softmax", name = 'classification')(x)

    return x

In [ ]:
def final(inputs):

    res_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(res_feature_extractor)

    return classification_output

In [ ]:
inputs = Input(shape=(299, 299, 3))
out_put = final(inputs)

model = Model(inputs, out_put)

219070464/219055592 [==============================] - 2s 0us/step


In [ ]:
# image_dim = 456*456
# model = tf.keras.models.Sequential([
#     # Note the input shape is the desired size of the image 150x150 with 3 bytes color
#     # This is the first convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(200, 200, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     # The second convolution
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The third convolution
#     tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The fourth convolution
#     tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The fifth convolution
#     #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     #tf.keras.layers.MaxPooling2D(2,2),
#     # Flatten the results to feed into a DNN
#     tf.keras.layers.Flatten(),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(512, activation='relu'),
#     # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import CategoricalAccuracy

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=[CategoricalAccuracy()])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/unzip_data/trainset'
batch_size = 32
epochs = 30
img_h_and_w = 299

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 30,
                                   width_shift_range = 0.15,
                                   height_shift_range = 0.15,

    shear_range=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_h_and_w, img_h_and_w),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_h_and_w, img_h_and_w),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs)

In [ ]:

model.save('classifier_model.h5') 
del model  


In [ ]:
from keras.models import load_model
model = load_model('classifier_model.h5')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

import cv2
import os

images = []
folder_path = '/content/test'

for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict(images, batch_size=10)
print(classes)

In [ ]:
import pandas as pd
names = []
for i in os.listdir(folder_path):
    names.append(i)

a = np.array(classes)
cc = np.argmax(a, axis=1)

prs = []
for i in cc:
    
    prs.append(i+1)


dic = {'name':names, 'predicated':prs}

df = pd.DataFrame(dic)
df.to_csv('results.csv', index = False)